# Capturing greenhouse gases with data

## Data Wrangling

### by Zachary Brown

The goal of this project was originally to merge two MOF databases to determine what chemical properties increase the CO2 capacity of a metal-organic framework (MOF). Those two databases only had 30 entries with the same MOF identifiers, so instead I will be using the [ARC MOF database](https://zenodo.org/record/7600474#.Y_ofvXbMKM8) which has over 200,000 theoretical MOFs and has both chemical properties and gas adsorption predictions included.

Some key terms that are used throughout this dataset and project include RDF - radial distribution functions (calculated for electronegativity, atomic hardness, van der Waals volume, dipole polarizability, atomic mass, and none), RAC - revised autocorrelations (calculated for electronegativity, nuclear charge, atom identity, connectivity and covalent radii), 

First we'll install the necessary libraries and import them.

In [1]:
!pip install numpy==1.24.2
!pip install pandas==1.5.3
!pip install requests==2.28.2
!pip install matplotlib==3.7.0

     ---------------------------------------- 7.6/7.6 MB 7.0 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 10.0 MB/s eta 0:00:00
     ------------------------------------- 965.4/965.4 kB 10.2 MB/s eta 0:00:00
     ---------------------------------------- 55.4/55.4 kB 1.5 MB/s eta 0:00:00
     ------------------------------------- 160.2/160.2 kB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 98.3/98.3 kB 5.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt

Now I'll start by downloading the topology dataset, which describes the geometric topology of the MOFs.

In [3]:
url = 'https://zenodo.org/record/7600474/files/all_topology_lists.csv?download=1'
r = requests.get(url, allow_redirects=True)
open('../data/raw/topology.csv', 'wb').write(r.content)

11683716

In [4]:
top = pd.read_csv('../data/raw/topology.csv')
top.head()

C:\Users\Zjbro\AppData\Local\Temp\ipykernel_6412\102696264.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  top = pd.read_csv('../data/raw/topology.csv')


,Name,filename,Crystalnet,likely topology
0,DB0-m12_o10_bcu.cif,bcu,bcu,bcu
1,DB0-m12_o12_bcu.cif,bcu,bcu,bcu
2,DB0-m12_o13_bcu.cif,bcu,bcu,bcu
3,DB0-m12_o14_bcu.cif,bcu,bcu,bcu
4,DB0-m12_o14_o22_f0_bcu.cif,bcu,bcu,bcu


In [5]:
top.shape

(264225, 4)

To join this dataframe with future ones I'll need to set the 'Name' column as the index, so I'll do that and then download the geometry dataset which has geometric properties of the MOFs.

In [6]:
top.set_index('Name', inplace=True)

In [7]:
url = 'https://zenodo.org/record/7600474/files/geometric_properties.csv?download=1'
r = requests.get(url, allow_redirects=True)
open('../data/raw/geom.csv', 'wb').write(r.content)

110395714

In [8]:
geo = pd.read_csv('../data/raw/geom.csv')
geo.head()

,Unnamed: 0,filename,UC_volume,Density,ASA,vASA,gASA,NASA,gNASA,vNASA,...,NPOAVA,NPOAVAf,NPOAVAg,Di,Df,Dif,ARC-MOF,DB_num,order_geo,bool_geo
0,0,DB0-m2_o1_o10_f0_pcu.sym.66.cif,901.788,1.233220,87.4832,970.108,786.644,0.0,0.0,0.0,...,0.0,0.0,0.0,5.41813,4.36524,5.39798,True,DB0,0,True
1,1,DB0-m28_o161_o113_f0_pts.cif,8183.190,0.389995,1749.5500,2137.970,5482.050,0.0,0.0,0.0,...,0.0,0.0,0.0,16.83322,15.07954,16.80076,False,DB0,1,True
2,2,DB1-Zn2O8N2-irmof20_A-irmof8_A_No13.cif,3853.140,0.652434,824.5020,2139.820,3279.750,0.0,0.0,0.0,...,0.0,0.0,0.0,11.24255,9.36124,11.24255,False,DB1,2,True
3,3,DB1-Zn4O13-BDC_A-irmof6_A_No267.cif,16975.800,0.815191,3234.8600,1905.570,2337.570,0.0,0.0,0.0,...,0.0,0.0,0.0,14.96430,6.83319,14.95745,False,DB1,3,True
4,4,DB0-m15_o27_aww.cif,236848.000,0.127610,17612.1000,743.601,5827.130,0.0,0.0,0.0,...,0.0,0.0,0.0,48.43682,38.41622,48.43682,False,DB0,4,True


In [9]:
geo.shape

(521316, 29)

In [10]:
geo.columns

Index(['Unnamed: 0', 'filename', 'UC_volume', 'Density', 'ASA', 'vASA', 'gASA',
       'NASA', 'gNASA', 'vNASA', 'AVA', 'AVAf', 'AVAg', 'NAVA', 'NAVAf',
       'NAVAg', 'POAVA', 'POAVAf', 'POAVAg', 'NPOAVA', 'NPOAVAf', 'NPOAVAg',
       'Di', 'Df', 'Dif', 'ARC-MOF', 'DB_num', 'order_geo', 'bool_geo'],
      dtype='object')

These column headers aren't particularly insightful, so I'm going to reference the journal article to rename these to something more useful.

In [11]:
geo.rename(columns={'UC_volume':'unit_cell_volume', 'ASA':'accessible_surface_area', 'vASA':'volumetric_surface_area',\
 'gASA':'gravimetric_surface_area', 'NASA':'inaccessible_surface_area', 'gNASA':'inac_grav_surf_area',\
 'vNASA':'inac_vol_surf_area', 'AVA':'accessible_volume_per_uc', 'AVAf':'volume_fraction', 'AVAg':'grav_volume',\
 'NAVA':'inac_vol', 'NAVAf':'inac_vol_frac', 'NAVAg':'inac_grav_vol', 'POAVA':'probe_occupiable_vol',\
 'POAVAf':'probe_occ_vol_frac', 'POAVAg':'grav_probe_occ_vol', 'NPOAVA':'inac_probe_occ_vol',\
 'NPOAVAf':'inac_probe_occ_vol_frac', 'NPOAVAg':'inac_probe_occ_grav_vol', 'Di':'largest_cav_diameter',\
 'Df':'pore_limiting_diameter', 'Dif':'largest_free_sphere_path_diam'},
           inplace=True)

In [12]:
geo.columns

Index(['Unnamed: 0', 'filename', 'unit_cell_volume', 'Density',
       'accessible_surface_area', 'volumetric_surface_area',
       'gravimetric_surface_area', 'inaccessible_surface_area',
       'inac_grav_surf_area', 'inac_vol_surf_area', 'accessible_volume_per_uc',
       'volume_fraction', 'grav_volume', 'inac_vol', 'inac_vol_frac',
       'inac_grav_vol', 'probe_occupiable_vol', 'probe_occ_vol_frac',
       'grav_probe_occ_vol', 'inac_probe_occ_vol', 'inac_probe_occ_vol_frac',
       'inac_probe_occ_grav_vol', 'largest_cav_diameter',
       'pore_limiting_diameter', 'largest_free_sphere_path_diam', 'ARC-MOF',
       'DB_num', 'order_geo', 'bool_geo'],
      dtype='object')

I don't care which database ARC-MOF drew these from, so I'm going to drop 'ARC-MOF' and 'DB_num'.

In [13]:
geo.drop(columns=['ARC-MOF', 'DB_num'], inplace=True)
geo.shape

(521316, 27)

Now I'll join the geometry and topology dataframes.

In [14]:
geo_top = geo.join(other = top, on = 'filename', how = 'inner', lsuffix='_geo', rsuffix='_top')

In [15]:
geo_top.shape

(263744, 31)

In [16]:
geo_top.head()

,filename,Unnamed: 0,filename_geo,unit_cell_volume,Density,accessible_surface_area,volumetric_surface_area,gravimetric_surface_area,inaccessible_surface_area,inac_grav_surf_area,...,inac_probe_occ_vol_frac,inac_probe_occ_grav_vol,largest_cav_diameter,pore_limiting_diameter,largest_free_sphere_path_diam,order_geo,bool_geo,filename_top,Crystalnet,likely topology
0,DB0-m2_o1_o10_f0_pcu.sym.66.cif,0,DB0-m2_o1_o10_f0_pcu.sym.66.cif,901.788,1.233220,87.4832,970.108,786.644,0.000000,0.000000,...,0.0,0.0,5.41813,4.36524,5.39798,0,True,pcu,pcu,pcu
6,DB0-m3_o23_o23_f0_pcu.sym.74.cif,6,DB0-m3_o23_o23_f0_pcu.sym.74.cif,7545.840,0.537679,1566.3300,2075.750,3860.570,0.000000,0.000000,...,0.0,0.0,10.43731,9.91429,10.43731,6,True,pcu,pcu,pcu
7,DB0-m2_o8_o25_f0_pcu.sym.91.cif,7,DB0-m2_o8_o25_f0_pcu.sym.91.cif,4172.230,0.371648,771.9300,1850.160,4978.270,0.000000,0.000000,...,0.0,0.0,12.93441,11.01397,12.93441,7,True,pcu,pcu,pcu
8,DB0-m29_o82_o46_f0_pts.sym.1.cif,8,DB0-m29_o82_o46_f0_pts.sym.1.cif,1715.110,0.786327,378.9050,2209.230,2809.550,0.000000,0.000000,...,0.0,0.0,8.35282,5.44658,7.30192,8,True,pts,pts,pts
10,DB0-m29_o99_o470_f0_pts.sym.128.cif,10,DB0-m29_o99_o470_f0_pts.sym.128.cif,2552.970,0.754924,419.5890,1643.530,2177.080,0.164038,0.642539,...,0.0,0.0,7.57868,4.51994,7.57868,10,True,pts,pts,pts


Let's clean it up a little and drop the 'Unnamed:0' and 'filename_geo' columns.

In [17]:
geo_top.drop(columns=['Unnamed: 0', 'filename_geo'])

,filename,unit_cell_volume,Density,accessible_surface_area,volumetric_surface_area,gravimetric_surface_area,inaccessible_surface_area,inac_grav_surf_area,inac_vol_surf_area,accessible_volume_per_uc,...,inac_probe_occ_vol_frac,inac_probe_occ_grav_vol,largest_cav_diameter,pore_limiting_diameter,largest_free_sphere_path_diam,order_geo,bool_geo,filename_top,Crystalnet,likely topology
0,DB0-m2_o1_o10_f0_pcu.sym.66.cif,901.788,1.233220,87.4832,970.108,786.644,0.000000,0.000000,0.000000,26.0256,...,0.0,0.0,5.41813,4.36524,5.39798,0,True,pcu,pcu,pcu
6,DB0-m3_o23_o23_f0_pcu.sym.74.cif,7545.840,0.537679,1566.3300,2075.750,3860.570,0.000000,0.000000,0.000000,2364.4100,...,0.0,0.0,10.43731,9.91429,10.43731,6,True,pcu,pcu,pcu
7,DB0-m2_o8_o25_f0_pcu.sym.91.cif,4172.230,0.371648,771.9300,1850.160,4978.270,0.000000,0.000000,0.000000,2102.0500,...,0.0,0.0,12.93441,11.01397,12.93441,7,True,pcu,pcu,pcu
8,DB0-m29_o82_o46_f0_pts.sym.1.cif,1715.110,0.786327,378.9050,2209.230,2809.550,0.000000,0.000000,0.000000,281.5860,...,0.0,0.0,8.35282,5.44658,7.30192,8,True,pts,pts,pts
10,DB0-m29_o99_o470_f0_pts.sym.128.cif,2552.970,0.754924,419.5890,1643.530,2177.080,0.164038,0.642539,0.851131,268.4700,...,0.0,0.0,7.57868,4.51994,7.57868,10,True,pts,pts,pts
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521302,DB0-m2_o12_o16_f0_pcu.sym.10.cif,1358.680,0.754709,290.7150,2139.690,2835.120,0.000000,0.000000,0.000000,189.8350,...,0.0,0.0,7.52873,5.74171,7.44496,100001,False,pcu,pcu,pcu
521304,DB0-m3_o160_o480_f0_fsc.sym.50.cif,1243.540,0.972493,216.3000,1739.390,1788.590,0.000000,0.000000,0.000000,154.4720,...,0.0,0.0,7.86425,5.25972,7.85850,100001,False,fsc,fsc,fsc
521310,DB0-m3_o7_o15_f0_pcu.sym.26.cif,3245.820,0.460190,607.9870,1873.140,4070.370,0.000000,0.000000,0.000000,1383.1700,...,0.0,0.0,14.76229,10.91728,14.76229,100001,False,pcu,pcu,pcu
521311,DB0-m2_o9_o11_f0_nbo.sym.43.cif,5025.910,0.784130,842.4600,1676.240,2137.700,0.000000,0.000000,0.000000,799.8230,...,0.0,0.0,9.80754,4.61436,9.34008,100001,False,nbo,nbo,nbo


Now that the two are merged I'll download the RDF dataset, which describes a wide range of chemical properties. 

In [ ]:
url = 'https://zenodo.org/record/7600474/files/RDFs.csv?download=1'
r = requests.get(url, allow_redirects=True)
open('../data/raw/rdf.csv', 'wb').write(r.content)

In [ ]:
rdf = pd.read_csv('../data/raw/rdf.csv')
rdf.head(10)

In [ ]:
rdf.shape

In [ ]:
rdf.columns

In [ ]:
rdf.set_index('Structure_Name', inplace=True)

I noticed that the structure names in this dataframe include 'repeat' in the names, which wasn't included in the other tables. I'm going to confirm that it's in all of the names, then remove it so I can join this dataframe to the geo_top.

In [ ]:
rdf[rdf.index.str.contains('.repeat.') == False]

In [ ]:
rdf.index = rdf.index.str.replace('_repeat', '')

In [ ]:
rdf.head()

In [ ]:
rdf.drop(columns='Unnamed: 0', inplace=True)

Time to join!

In [ ]:
geo_top_rdf = geo_top.join(other = rdf, on = 'filename', how='inner', rsuffix='rdf')
geo_top_rdf.shape

In [ ]:
geo_top_rdf.head()

In [ ]:
geo_top_rdf.set_index('filename', inplace=True)
geo_top_rdf.drop(columns=['Unnamed: 0', 'filename_geo'], inplace = True)

Next I'll download the process datafile, which has adsorption data for five different gas separation processes: natural gas purification (90% CH4/10% CO2), post-combustion VSA (17% CO2/83% N2), pre-combustion PSA (40% CO2/60% H2), landfill gas VPSA (42-96% CO2/58-4% CH4) and methane storage PSA (100% CH4). 

In [ ]:
url = 'https://zenodo.org/record/7600474/files/overall_process.csv?download=1'
r = requests.get(url, allow_redirects=True)
open('../data/raw/process.csv', 'wb').write(r.content)

In [ ]:
process = pd.read_csv('../data/raw/process.csv')
process.head()

In [ ]:
process.shape

Again I'm noticing the 'repeat' in the filename, so I'll remove that so I can join the dataframes.

In [ ]:
process['filename'] = process['filename'].str.replace('_repeat', '.cif')

In [ ]:
process.set_index('filename', inplace=True)

Fortunately, this database has data for exactly the process I'm interested in: post-combustion VSA, where CO2 is removed from power plant exhaust. I'll cut down the number of rows now to preserve memory and make it easier to work with.

In [ ]:
pcv = process[process['process'] == 'post-combustion-vsa']

In [ ]:
pcv.head()

Now I'll merge the post-combustion VSA data into the rest.

In [ ]:
merged = geo_top_rdf.join(other = pcv, on = 'filename', how='inner', rsuffix = 'process')

In [ ]:
merged.shape

My last addition will be the RACs, which describe properties around the ligands and metal centers.

In [ ]:
url = 'https://zenodo.org/record/7600474/files/RACs.csv?download=1'
r = requests.get(url, allow_redirects=True)
open('../data/raw/racs.csv', 'wb').write(r.content) 

In [ ]:
racs = pd.read_csv('../data/raw/racs.csv')
racs.head()

In [ ]:
racs.shape

In [ ]:
print(racs.columns.tolist())

I'll drop a few unnecessary columns.

In [ ]:
racs.drop(columns=['ARC_MOF', 'DB_num', 'Unnamed: 0'], inplace=True)
racs.set_index('filename', inplace=True)

In [ ]:
racs.head()

In [ ]:
merged.head()

Now I'll join RACs to the merged dataframe.

In [ ]:
total = merged.join(other = racs, on = 'filename', how = 'inner', rsuffix = 'racs')
total.shape

In [ ]:
total.head()

Ok, time to start cleaning. I'll check the datatypes of the columns to start.

In [ ]:
print(total.columns.to_series().groupby(total.dtypes).groups)

First of all, I'm seeing some more columns I need to remove.

In [ ]:
trimmed = total.drop(columns=['filename_top', 'process'])

Now I'm not seeing any obviously incorrect datatypes, so I'll check to see if I have any columns that are missing too much data.

In [ ]:
trimmed.head()

In [ ]:
print((trimmed.isna().sum()/len(trimmed)).sort_values(ascending=False))

Ok, missing data isn't a problem in this dataset. Let me take a look at how much of a columns values are 0. I've got ~50k rows and 899 columns, so I will need to reduce the features of this dataset. I suspect there are some mostly-zero columns I can drop.

In [ ]:
zeroes = {}

for col in trimmed.columns:
    if trimmed[col].dtypes == 'int64':
        if 0 in trimmed[col].unique():
            zeroes[col] = trimmed[col].value_counts(normalize=True)[0]
    
sorted_zeroes = sorted(zeroes.items(), key = lambda x:x[1])
print(sorted_zeroes)

Wow, so none of the columns have many zero values either. This is a surprisingly clean dataset. Since there are already over 900 columns I'll have to check for values that are unexpected by going through each group of columns (for example, RDFs of electronegativity) and make a loop that calls out anything outside of an expected range. RDFs cover electronegativity, atomic hardness, van der Waals volume, dipole polarizability, atomic mass, and none, so I'll break each of them out and see if any invalid values are present.

In [ ]:
# Electronegativity should range betwen 0 and 4
for col in trimmed.columns:
    if 'RDF_electronegativity' in col:
        if min(trimmed[col]) < 0 or max(trimmed[col]) > 4:
            print(col)

In [ ]:
# Atomic hardness ranges between 0 and 13
for col in trimmed.columns:
    if 'RDF_hardness' in col:
        if min(trimmed[col]) < 0 or max(trimmed[col]) > 13:
            print(col)

In [ ]:
# van der Waals volume should be greater than zero
for col in trimmed.columns:
    if 'RDF_v' in col:
        if min(trimmed[col]) < 0:
            print(col)

In [ ]:
# Dipole polarizability should be non-negative
for col in trimmed.columns:
    if 'RDF_polarizability' in col:
        if min(trimmed[col]) < 0:
            print(col)

In [ ]:
# Atomic mass must be greater than zero
for col in trimmed.columns:
    if 'RDF_mass' in col:
        if min(trimmed[col]) < 0:
            print(col)

Ok, all of the RDF values meet expectations. This seems to be a very clean dataset. Given that the values are provided by theoretical calculations, it seems reasonable to expect the data to be within reasonable bounds and already cleaned for the analysis the authors did. I'll save it as-is and move on to exploratory data analysis.

In [ ]:
trimmed.to_csv('../data/interim/wrangled.csv')